In [ ]:
%load_ext autoreload
%autoreload 2

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import pandas as pd
import json
import folium
from folium import Map, Polygon
from shapely import intersection_all
from shapely.geometry import mapping
import shapely

import numpy as np

In [200]:
# aoi = Vector('/data/aoi_bretagne.geojson')
# infiles = [
#     "/data/S1/S1A_IW_SLC__1SDV_20230826T180527_20230826T180554_050050_0605B2_E768.zip",
#     "/data/S1/S1A_IW_SLC__1SDV_20230814T180526_20230814T180553_049875_05FFB4_7D92.zip",
# ]
file_aoi = "/data/aoi_bretagne.geojson"
aoi = geojson_to_wkt(read_geojson(file_aoi)["features"][0])
with open("/data/creds.json") as f:
    creds = json.load(f)
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    producttype="SLC",
    date=("20220110", "20220430"),
    #  orbitdirection='ASCENDING',
    limit=50,
)
gdf = api.to_geodataframe(products).reset_index()
# gdf_ = gdf.loc[:, ["title", "geometry", "summary", "relativeorbitnumber", "slicenumber"]].reset_index()

### TODO
- NO: group only by orbit
- compute overlap in %
- NO: form pairs that overlap > n%
- OR: just color code by orbit
- Show intersection by slice and display all products with indices
- add overlap checking parameter in processor

In [267]:
map_ = Map()
grouped = gdf.groupby(["relativeorbitnumber", "slicenumber"])
groups = list(grouped.groups.keys())
ll = gdf.total_bounds.reshape(2,2)
bbox = list(map(lambda x: [x[1], x[0]], ll))
for g in groups:
    # convert Multipolygons to Polygons
    dfg = grouped.get_group(g).explode(index_parts=False).to_crs("EPSG:4236")
    
    # find the common part of all geometries
    inter = intersection_all(dfg.geometry)
    folium_geo = folium.GeoJson(
        mapping(inter), style_function=lambda x: {"fillColor": "lightblue"}
    )
    date_ts = dfg["beginposition"].dt.strftime("%Y-%m-%d %X")
    date_str = "<br>".join(date_ts.index.astype(str).str.cat(date_ts.values, sep=": "))
    folium.Tooltip(date_str).add_to(folium_geo)
    folium_geo.add_to(map_)
    # print(inter)
map_.fit_bounds(bbox)
map_

In [192]:
" ".join(date_ts.reset_index(drop=True))

'2022-04-13 06:24:31 2022-04-01 06:24:31 2022-03-20 06:24:31 2022-03-08 06:24:30 2022-02-24 06:24:31 2022-02-12 06:24:31 2022-01-31 06:24:31 2022-01-19 06:24:31'

In [233]:
dfg.centroid

/tmp/ipykernel_10311/1540877790.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dfg.centroid


8     POINT (-3.53204 47.46611)
13    POINT (-3.53281 47.46597)
20    POINT (-3.53231 47.46641)
26    POINT (-3.53076 47.46567)
32    POINT (-3.53064 47.46579)
37    POINT (-3.53066 47.46618)
44    POINT (-3.53108 47.46628)
49    POINT (-3.53062 47.46610)
dtype: geometry

In [223]:
f'{date_ts.index.astype(str)} {date_ts.values}'

"Index(['8', '13', '20', '26', '32', '37', '44', '49'], dtype='object') ['2022-04-13 06:24:31' '2022-04-01 06:24:31' '2022-03-20 06:24:31'\n '2022-03-08 06:24:30' '2022-02-24 06:24:31' '2022-02-12 06:24:31'\n '2022-01-31 06:24:31' '2022-01-19 06:24:31']"

In [231]:
"<br>".join(date_ts.index.astype(str).str.cat(date_ts.values, sep=": "))

'8: 2022-04-13 06:24:31<br>13: 2022-04-01 06:24:31<br>20: 2022-03-20 06:24:31<br>26: 2022-03-08 06:24:30<br>32: 2022-02-24 06:24:31<br>37: 2022-02-12 06:24:31<br>44: 2022-01-31 06:24:31<br>49: 2022-01-19 06:24:31'

In [259]:
ll = gdf.total_bounds.reshape(2,2)
list(map(lambda x: [x[1], x[0]], ll))

[[46.450706, -5.454467], [50.142849, 0.566494]]